In [1]:
import tensorflow as tf
x=tf.Variable(3,name='x')
y=tf.Variable(4, name='y')
f=x*x*y+y+2
init=tf.global_variables_initializer()


In [2]:
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)
sess.close()

42


In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result=f.eval()

In [9]:
with tf.Session() as sess:
    init.run()
    result=f.eval()
    print(result)

42


# Linear regression on California housing dataset

## Normal equations:

In [2]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing()
m,n=housing.data.shape

housing_data_plus_bias=np.c_[np.ones((m,1)), housing.data]

X=tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name='y')
XT=tf.transpose(X)

theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value=theta.eval()

print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


## Gradient descent

Need to scale the data first for better convergence

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data=scaler.fit_transform(housing.data)

scaled_housing_data_plus_bias=np.c_[np.ones((m,1)), scaled_housing_data]


In [17]:
nloops=1000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

gradients=2/m*tf.matmul(tf.transpose(X),error)

training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        if loop%100==0:
            print('loop', loop, 'MSE=', mse.eval())
        sess.run(training_op)
        
    best_theta=theta.eval()
    
    print(best_theta)

loop 0 MSE= 4.85334
loop 100 MSE= 0.747335
loop 200 MSE= 0.668665
loop 300 MSE= 0.639556
loop 400 MSE= 0.617165
loop 500 MSE= 0.599403
loop 600 MSE= 0.585237
loop 700 MSE= 0.573893
loop 800 MSE= 0.564771
loop 900 MSE= 0.557409
[[ 2.06855249]
 [ 0.97460502]
 [ 0.16666   ]
 [-0.50069165]
 [ 0.48311004]
 [ 0.01104329]
 [-0.04647031]
 [-0.4144997 ]
 [-0.40015447]]


using autodiff:

In [18]:
nloops=1000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#difference is here
gradients=tf.gradients(mse,[theta])[0]

training_op=tf.assign(theta,theta-learning_rate*gradients)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        if loop%100==0:
            print('loop', loop, 'MSE=', mse.eval())
        sess.run(training_op)
        
    best_theta=theta.eval()
    
    print(best_theta)

loop 0 MSE= 10.9403
loop 100 MSE= 0.73624
loop 200 MSE= 0.533643
loop 300 MSE= 0.526731
loop 400 MSE= 0.525957
loop 500 MSE= 0.525583
loop 600 MSE= 0.52531
loop 700 MSE= 0.525099
loop 800 MSE= 0.524938
loop 900 MSE= 0.524811
[[ 2.06855226]
 [ 0.84255821]
 [ 0.12524015]
 [-0.28237295]
 [ 0.31628352]
 [-0.00231135]
 [-0.04015082]
 [-0.84042633]
 [-0.81221974]]


Using an optimizer

In [7]:
nloops=5000
learning_rate=0.01

X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name='X')
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name='y')
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#use an optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        if loop%500==0:
            print('loop', loop, 'MSE=', mse.eval())
        sess.run(training_op)
        
    best_theta=theta.eval()
    
    print(best_theta)

loop 0 MSE= 3.26923
loop 500 MSE= 0.561195
loop 1000 MSE= 0.537394
loop 1500 MSE= 0.529233
loop 2000 MSE= 0.526227
loop 2500 MSE= 0.525074
loop 3000 MSE= 0.524621
loop 3500 MSE= 0.524441
loop 4000 MSE= 0.524369
loop 4500 MSE= 0.52434
[[ 2.06855249]
 [ 0.83274233]
 [ 0.11935294]
 [-0.27139792]
 [ 0.31053811]
 [-0.0043248 ]
 [-0.03944373]
 [-0.8925699 ]
 [-0.86358738]]


## Mini-batch Gradient descent

In [6]:
A=tf.placeholder(tf.float32, shape=(None,3))
B=A+5
with tf.Session() as sess:
    B_val_1=B.eval(feed_dict={A: [[1,2,3]]})
    B_val_2=B.eval(feed_dict={A: [[4,5,6], [7,8,9]]})

    
print(B_val_1)
print(B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [7]:
nloops=10
learning_rate=0.01

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data=scaler.fit_transform(housing.data)

scaled_housing_data_plus_bias=np.c_[np.ones((m,1)), scaled_housing_data]

import pandas as pd
df=pd.DataFrame(np.c_[housing.target, scaled_housing_data], columns=['y']+housing.feature_names)
df['Bias']=np.ones((m,1))

cols=list(df.columns.values)
cols_temp=cols[1:len(cols)-1]

cols[1]='Bias'
cols[2:]=cols_temp

df=df[cols]


df.to_csv('data.csv',)

batch_size=100
n_batches=int(np.ceil(m/batch_size))

#Placeholder to feed the minibatches into X and y
X=tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y=tf.placeholder(tf.float32, shape=(None, 1), name='y')

theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')
y_pred=tf.matmul(X,theta, name='prediction')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#use an optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

init=tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        for batch_index in range(n_batches):
            #get the corresponding minibatch
            data_batch=df.iloc[batch_index*batch_size: min((batch_index+1)*batch_size, m)]
            y_batch=np.asarray(data_batch.y).reshape((-1,1))
            X_batch=np.asarray(data_batch[data_batch.columns[1:]])
            #and feed it to the training operation
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if loop%5==0:
            #the eval of mse requires X and y
            print('loop', loop, 'MSE=', mse.eval(feed_dict={X: X_batch, y: y_batch }))
    
    best_theta=theta.eval()
    
    print(best_theta)
    
    


loop 0 MSE= 0.133957
loop 5 MSE= 0.125182
[[ 2.00519872]
 [ 0.79733837]
 [ 0.13927931]
 [-0.25245261]
 [ 0.2886897 ]
 [-0.00551016]
 [-0.01423678]
 [-0.83756173]
 [-0.86223739]]


## Saving the graph

In [13]:
#Parameters for the gradient descent
nloops=50
learning_rate=0.01

#Preprocessing of the data to improve convergence of grandient descent
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data=scaler.fit_transform(housing.data)

#add a bias unit
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)), scaled_housing_data]

#tranform the data into a pandas dataframe to save it for later, and reorganize the columns
import pandas as pd
df=pd.DataFrame(np.c_[housing.target, scaled_housing_data], columns=['y']+housing.feature_names)
df['Bias']=np.ones((m,1))

cols=list(df.columns.values)
cols_temp=cols[1:len(cols)-1]

cols[1]='Bias'
cols[2:]=cols_temp

df=df[cols]

df.to_csv('data.csv',)


#parameter for the minibatch gradient descent
batch_size=100
n_batches=int(np.ceil(m/batch_size))


#define first the different nodes of the graph
#Placeholder to feed the minibatches into X and y
X=tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y=tf.placeholder(tf.float32, shape=(None, 1), name='y')

#the variable to train
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')

#the prediction
y_pred=tf.matmul(X,theta, name='prediction')

#the error and the function to minimize
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#use an optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

#initializer
init=tf.global_variables_initializer()

#Saver
saver=tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        if loop%5 ==0: 
            save_path=saver.save(sess,'tmp/my_model.ckpt')
        for batch_index in range(n_batches):
            #get the corresponding minibatch
            data_batch=df.iloc[batch_index*batch_size: min((batch_index+1)*batch_size, m)]
            y_batch=np.asarray(data_batch.y).reshape((-1,1))
            X_batch=np.asarray(data_batch[data_batch.columns[1:]])
            #and feed it to the training operation
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if loop%5==0:
            #the eval of mse requires X and y
            print('loop', loop, 'MSE=', mse.eval(feed_dict={X: X_batch, y: y_batch }))
    
    best_theta=theta.eval()
    print(best_theta)
    save_path=saver.save(sess,'tmp/my_model_final.ckpt')

loop 0 MSE= 0.203762
loop 5 MSE= 0.124818
loop 10 MSE= 0.123936
loop 15 MSE= 0.124325
loop 20 MSE= 0.124396
loop 25 MSE= 0.124398
loop 30 MSE= 0.124393
loop 35 MSE= 0.12439
loop 40 MSE= 0.124389
loop 45 MSE= 0.124388
[[ 2.00290966]
 [ 0.78347826]
 [ 0.13612111]
 [-0.23014921]
 [ 0.27157745]
 [-0.00706323]
 [-0.01321218]
 [-0.86899805]
 [-0.89598686]]


Importation of a model from a checkpoint

In [14]:
saver=tf.train.import_meta_graph('tmp/my_model_final.ckpt.meta')
    
with tf.Session() as sess:
    saver.restore(sess,'tmp/my_model_final.ckpt')
    for loop in range(nloops):
        for batch_index in range(n_batches):
            #get the corresponding minibatch
            data_batch=df.iloc[batch_index*batch_size: min((batch_index+1)*batch_size, m)]
            y_batch=np.asarray(data_batch.y).reshape((-1,1))
            X_batch=np.asarray(data_batch[data_batch.columns[1:]])
            #and feed it to the training operation
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if loop%5==0:
            #the eval of mse requires X and y
            print('loop', loop, 'MSE=', mse.eval(feed_dict={X: X_batch, y: y_batch }))
    
    best_theta=theta.eval()
    print(best_theta)

INFO:tensorflow:Restoring parameters from tmp/my_model_final.ckpt
loop 0 MSE= 0.124388
loop 5 MSE= 0.124388
loop 10 MSE= 0.124388
loop 15 MSE= 0.124388
loop 20 MSE= 0.124388
loop 25 MSE= 0.124388
loop 30 MSE= 0.124388
loop 35 MSE= 0.124388
loop 40 MSE= 0.124388
loop 45 MSE= 0.124388
[[ 2.00290895]
 [ 0.78347152]
 [ 0.13612019]
 [-0.23013733]
 [ 0.27156752]
 [-0.00706378]
 [-0.01321162]
 [-0.86900991]
 [-0.89599925]]


## Using TensorBoard

In [35]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()
df=pd.read_csv('data.csv', index_col=0, header=0)
df_X=df.drop('y', axis=1)
df_y=df.y


def fetch_batch(loop, batch_index, batch_size):
    np.random.seed(loop * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = np.asarray(df_X.iloc[indices])
    y_batch = np.asarray(df_y.iloc[indices]).reshape((-1,1))
    return X_batch, y_batch

from datetime import datetime

now=datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir='tf_logs'
logdir='{}/run-{}/'.format(root_logdir, now)


#parameter for the minibatch gradient descent
batch_size=100
n_batches=int(np.ceil(m/batch_size))

#Parameters for the gradient descent
nloops=10
learning_rate=0.01


#define first the different nodes of the graph
#Placeholder to feed the minibatches into X and y
X=tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y=tf.placeholder(tf.float32, shape=(None, 1), name='y')

#the variable to train
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')

#the prediction
y_pred=tf.matmul(X,theta, name='prediction')

#the error and the function to minimize
error=y_pred-y
mse=tf.reduce_mean(tf.square(error, name='mse'))

#use an optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

#initializer
init=tf.global_variables_initializer()

mse_summary=tf.summary.scalar('MSE', mse)
file_writer=tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        for batch_index in range(n_batches):
            X_batch, y_batch=fetch_batch(loop, batch_index, batch_size)
            if batch_index % 10==0:
                summary_str=mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                step=loop*n_batches+batch_index
                file_writer.add_summary(summary_str,step)
            #get the corresponding minibatch
            #and feed it to the training operation
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
file_writer.close()

print(best_theta)

[[ 2.07013464]
 [ 0.83224338]
 [ 0.11718354]
 [-0.25360975]
 [ 0.33584762]
 [ 0.00311589]
 [-0.01102609]
 [-0.90628386]
 [-0.87120152]]


## Name Scopes

In [36]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
reset_graph()
df=pd.read_csv('data.csv', index_col=0, header=0)
df_X=df.drop('y', axis=1)
df_y=df.y


def fetch_batch(loop, batch_index, batch_size):
    np.random.seed(loop * n_batches + batch_index)  
    indices = np.random.randint(m, size=batch_size)  
    X_batch = np.asarray(df_X.iloc[indices])
    y_batch = np.asarray(df_y.iloc[indices]).reshape((-1,1))
    return X_batch, y_batch

from datetime import datetime

now=datetime.utcnow().strftime('%Y%m%d%H%M%S')
root_logdir='tf_logs'
logdir='{}/run-{}/'.format(root_logdir, now)


#parameter for the minibatch gradient descent
batch_size=100
n_batches=int(np.ceil(m/batch_size))

#Parameters for the gradient descent
nloops=10
learning_rate=0.01


#define first the different nodes of the graph
#Placeholder to feed the minibatches into X and y
X=tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y=tf.placeholder(tf.float32, shape=(None, 1), name='y')

#the variable to train
theta=tf.Variable(tf.random_uniform([n+1,1],-1.,1.), name='theta')

#the prediction
y_pred=tf.matmul(X,theta, name='prediction')

#the error and the function to minimize
with tf.name_scope('loss') as scope:
    error=y_pred-y
    mse=tf.reduce_mean(tf.square(error, name='mse'))

#use an optimizer
optimizer=tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)

#initializer
init=tf.global_variables_initializer()

mse_summary=tf.summary.scalar('MSE', mse)
file_writer=tf.summary.FileWriter(logdir, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(init)
    
    for loop in range(nloops):
        for batch_index in range(n_batches):
            X_batch, y_batch=fetch_batch(loop, batch_index, batch_size)
            if batch_index % 10==0:
                summary_str=mse_summary.eval(feed_dict={X:X_batch, y:y_batch})
                step=loop*n_batches+batch_index
                file_writer.add_summary(summary_str,step)
            #get the corresponding minibatch
            #and feed it to the training operation
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()
file_writer.close()

print(best_theta)

[[ 2.07013464]
 [ 0.83224338]
 [ 0.11718354]
 [-0.25360975]
 [ 0.33584762]
 [ 0.00311589]
 [-0.01102609]
 [-0.90628386]
 [-0.87120152]]


## Modularity

In [55]:
def relu(X):
    with tf.name_scope('relu'): 
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal(w_shape), name='weights')
        b=tf.Variable(0.0, name='bias')
        z=tf.add(tf.matmul(X,w),b, name='z')
        return tf.maximum(z, 0., name='relu')

reset_graph()

n_features=3
X=tf.placeholder(dtype=tf.float32, shape=(None,n_features), name='X')
relus=[relu(X) for i in range(5)]
output=tf.add_n(relus, name='output')

file_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())
file_writer.close()

